In [85]:
#imports

import shutil

import cv2
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import split_folders
import sklearn 


In [5]:
#Defining the paths for images

path_ASL_alphabet_train = "D:\Github\MAIS202\AmericanSignLanguage-Translator\Images\asl-alphabet\asl_alphabet_train\asl_alphabet_train"
path_ASL_alphabet_test = "D:\Github\MAIS202\AmericanSignLanguage-Translator\Images\asl-alphabet\asl_alphabet_test\asl_alphabet_test"

#Note: TODO Dataset for digits is saved as a .npy file. Will be done for next deliverable



In [6]:


# # Loading images from a folder
# def load_images (folder):
#     images = []
#     for image_name in os.listdir(folder):
#         img = cv2.imread(os.path.join(folder,image_name))
#         if img is not None:
#             images.append(img)
    
#     return images


In [33]:
train_dir ='D:\\Github\\MAIS202\\AmericanSignLanguage-Translator\\Images\\asl-alphabet\\asl_alphabet_train\\train'
sample_path = 'D:\\Github\\MAIS202\\AmericanSignLanguage-Translator\\Images\\asl-alphabet\\asl_alphabet_train\\sample_split'


for folder_name in os.listdir(sample_path):
    mk_dir_path = os.path.join(sample_path,folder_name)
       
    shutil.rmtree(mk_dir_path)

In [36]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure







train_dir ='D:\\Github\\MAIS202\\AmericanSignLanguage-Translator\\Images\\asl-alphabet\\asl_alphabet_train\\train'
sample_path = 'D:\\Github\\MAIS202\\AmericanSignLanguage-Translator\\Images\\asl-alphabet\\asl_alphabet_train\\sample'


##

for folder_name in os.listdir(train_dir):
    mk_dir_path = os.path.join(sample_path,folder_name)
    
    
    os.mkdir(mk_dir_path)
    
    
    for image_name in os.listdir(train_dir+'\\'+ folder_name)[::10]:
        
        
        img = cv2.imread(os.path.join(train_dir+'\\'+ folder_name,image_name) )
        
        
        cv2.imwrite(os.path.join(mk_dir_path,image_name)+'.jpg',img)



FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:\\Github\\MAIS202\\AmericanSignLanguage-Translator\\Images\\asl-alphabet\\asl_alphabet_train\\sample\\A'

In [37]:
data_dir = 'D:\\Github\\MAIS202\\AmericanSignLanguage-Translator\\Images\\asl-alphabet\\asl_alphabet_train\\sample_split'

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 29

# Batch size for training (change depending on how much memory you have)
batch_size = 20

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True


In [38]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [39]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [40]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [41]:
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),    
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [42]:
device

device(type='cuda', index=0)

In [43]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [44]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/14
----------
train Loss: 3.1868 Acc: 0.1248
val Loss: 2.7891 Acc: 0.2802

Epoch 1/14
----------
train Loss: 2.6369 Acc: 0.3297
val Loss: 2.2503 Acc: 0.4432

Epoch 2/14
----------
train Loss: 2.3239 Acc: 0.4330
val Loss: 1.9113 Acc: 0.5366

Epoch 3/14
----------
train Loss: 2.1137 Acc: 0.4854
val Loss: 1.7860 Acc: 0.5237

Epoch 4/14
----------
train Loss: 1.9550 Acc: 0.5165
val Loss: 1.5774 Acc: 0.5776

Epoch 5/14
----------
train Loss: 1.8637 Acc: 0.5362
val Loss: 1.4393 Acc: 0.6343

Epoch 6/14
----------
train Loss: 1.7716 Acc: 0.5639
val Loss: 1.3659 Acc: 0.6473

Epoch 7/14
----------
train Loss: 1.6930 Acc: 0.5843
val Loss: 1.2835 Acc: 0.6681

Epoch 8/14
----------
train Loss: 1.6395 Acc: 0.5845
val Loss: 1.2109 Acc: 0.6889

Epoch 9/14
----------
train Loss: 1.5922 Acc: 0.5852
val Loss: 1.1829 Acc: 0.6990

Epoch 10/14
----------
train Loss: 1.5499 Acc: 0.6037
val Loss: 1.1549 Acc: 0.6868

Epoch 11/14
----------
train Loss: 1.5333 Acc: 0.6025
val Loss: 1.0889 Acc: 0.7119

Ep

In [77]:
def eval_model(model, dataloaders):

    ids = np.array([])
    pred = np.array([])

    # Each epoch has a training and validation phase
    for phase in ['test']:
        model.eval()   # Set model to evaluate mode
        
        running_corrects = 0
        
        # Iterate over data.
        for i, (inputs, labels) in enumerate(dataloaders[phase]):

            inputs = inputs.to(device)
            labels = labels.to(device)
            # print(i)
            # print(dataloaders[phase].dataset.samples[i])
            # print(dataloaders[phase].dataset.samples[i][0].split("/")[3].split(".")[0])

            # zero the parameter gradients

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)

                _, preds = torch.max(outputs, 1)
                
                ids = np.concatenate( (ids, labels.cpu().numpy() ), axis=None)
                
                pred = np.concatenate((pred, preds.cpu().numpy()), axis=None)
                
                # print(preds.cpu().numpy())
                # print(labels.cpu().data.numpy())
            # statistics
            

    return model, ids, pred

In [78]:
# Create training and validation datasets
image_datasets_test = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['val', 'test']}
# Create training and validation dataloaders
dataloaders_dict_test = {x: torch.utils.data.DataLoader(image_datasets_test[x], batch_size=1, shuffle=False, num_workers=4) for x in ['val', 'test']}


In [79]:
_, ids, labels = eval_model(model_ft, dataloaders_dict_test)


In [82]:
print(ids[::10])
print(labels[::10])

[ 0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  2.  2.  2.  2.  2.  3.  3.  3.
  3.  3.  4.  4.  4.  4.  5.  5.  5.  5.  5.  6.  6.  6.  6.  6.  7.  7.
  7.  7.  7.  8.  8.  8.  8.  8.  9.  9.  9.  9. 10. 10. 10. 10. 10. 11.
 11. 11. 11. 11. 12. 12. 12. 12. 12. 13. 13. 13. 13. 13. 14. 14. 14. 14.
 15. 15. 15. 15. 15. 16. 16. 16. 16. 16. 17. 17. 17. 17. 17. 18. 18. 18.
 18. 18. 19. 19. 19. 19. 20. 20. 20. 20. 20. 21. 21. 21. 21. 21. 22. 22.
 22. 22. 22. 23. 23. 23. 23. 23. 24. 24. 24. 24. 25. 25. 25. 25. 25. 26.
 26. 26. 26. 26. 27. 27. 27. 27. 27. 28. 28. 28. 28. 28.]
[ 0.  0.  4.  0.  0.  1.  4.  1.  1.  1.  2. 26.  2.  2.  2.  3.  3.  3.
  3.  3.  4. 25. 12.  4.  5.  5.  5. 10.  5.  7.  6.  7.  6.  6.  7.  7.
  7.  7.  7.  8. 11.  9.  8. 10. 25. 25.  9.  7. 10. 10. 17. 10. 10.  9.
 19. 11. 11. 11. 13. 12. 12. 13. 12. 13.  7. 13. 13. 13. 14.  7. 14. 14.
 15. 15. 16. 15. 15. 14. 16. 16. 16. 16. 17. 17. 17. 17. 17. 18.  4. 18.
 18. 18. 19.  4. 19.  6. 20. 22. 20. 17. 17.  6. 22. 21. 10. 17. 2

In [96]:
from sklearn import metrics

print (metrics.confusion_matrix(ids,labels) )

print (metrics.classification_report(ids,labels, digits=4))

[[36  0  0  0  6  1  1  1  0  1  1  0  0  0  0  0  0  0  1  0  0  0  0  0
   0  0  0  0  0]
 [ 1 33  0  0  5  0  0  0  0  2  1  0  0  0  0  0  0  1  0  0  5  0  0  0
   0  0  0  0  0]
 [ 0  0 40  0  0  2  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
   0  1  3  0  0]
 [ 0  0  0 38  0  3  0  0  2  2  0  0  0  0  0  0  0  2  0  0  0  0  0  1
   0  0  0  0  0]
 [ 2  0  0  0 39  1  1  1  0  0  0  0  1  0  0  0  0  0  1  1  0  0  0  0
   0  1  0  0  0]
 [ 0  0  0  0  0 40  0  1  0  1  2  1  0  0  0  0  0  0  0  1  0  2  0  0
   0  0  0  0  0]
 [ 0  0  0  0  0  0 34 10  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0]
 [ 0  0  0  0  0  0  2 46  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0]
 [ 1  0  0  0  4  0  1  0 23  4  6  2  0  0  0  0  0  5  0  0  0  0  0  1
   0  1  0  0  0]
 [ 0  0  0  0  0  1  1  2  1 38  0  0  0  0  0  0  0  0  1  0  0  0  0  0
   1  2  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0 44  0  0  0  0  0  0  4  0  0  0  0  0  0
   0  